### Задание 2
Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления.

In [128]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

from IPython.display import Image
from IPython.core.display import HTML 

In [223]:
#исходная таблица
data = pd.read_csv('таблица.csv')
data.rename(columns={'Unnamed: 0': 'text', 'Unnamed: 1': 'я', 'Unnamed: 2': 'ты', 'Unnamed: 3': 'и', 'Unnamed: 4' : 'только', 'Unnamed: 5' : 'не', 'Unnamed: 6' : 'он'}, inplace=True)

data.head()

,text,я,ты,и,только,не,он
0,я и ты,1,1,1,0,0,0
1,ты и я,1,1,1,0,0,0
2,я я и только я,3,0,1,1,0,0
3,только не я,1,0,0,1,1,0
4,он,0,0,0,0,0,1


In [198]:
f = open("таблица1.txt", 'r', encoding='utf8')
corpus = f.read().splitlines()
doc1, doc2, doc3, doc4, doc5 = corpus
doc1 = doc1.split(' ')
doc2 = doc2.split(' ')
doc3 = doc3.replace(',', '').split(' ')
doc4 = doc4.split(' ')
doc5 = doc5.split(' ')

uniqueWords = set(doc1).union(set(doc2)).union(set(doc3)).union(set(doc4)).union(set(doc5))
numOfWords1 = dict.fromkeys(uniqueWords, 0)
for word in doc1:
    numOfWords1[word] += 1
numOfWords2 = dict.fromkeys(uniqueWords, 0)
for word in doc2:
    numOfWords2[word] += 1
numOfWords3 = dict.fromkeys(uniqueWords, 0)
for word in doc3:
    numOfWords3[word] += 1
numOfWords4 = dict.fromkeys(uniqueWords, 0)
for word in doc4:
    numOfWords4[word] += 1
numOfWords5 = dict.fromkeys(uniqueWords, 0)
for word in doc5:
    numOfWords5[word] += 1

In [199]:
print(uniqueWords)

{'и', 'ты', 'я', 'он', 'не', 'только'}


In [197]:
print(numOfWords1)
print(numOfWords2)
print(numOfWords3)
print(numOfWords4)
print(numOfWords5)

{'и': 1, 'ты': 1, 'я': 1, 'он': 0, 'не': 0, 'только': 0}
{'и': 1, 'ты': 1, 'я': 1, 'он': 0, 'не': 0, 'только': 0}
{'и': 1, 'ты': 0, 'я': 3, 'он': 0, 'не': 0, 'только': 1}
{'и': 0, 'ты': 0, 'я': 1, 'он': 0, 'не': 1, 'только': 1}
{'и': 0, 'ты': 0, 'я': 0, 'он': 1, 'не': 0, 'только': 0}


In [200]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

tf1 = computeTF(numOfWords1, doc1)
tf2 = computeTF(numOfWords2, doc2)
tf3 = computeTF(numOfWords3, doc3)
tf4 = computeTF(numOfWords4, doc4)
tf5 = computeTF(numOfWords5, doc5)

In [214]:
print('Term frequency in doc1:', tf1)
print('Term frequency in doc2:', tf2)
print('Term frequency in doc3:', tf3)
print('Term frequency in doc4:', tf4)
print('Term frequency in doc5:', tf5)

Term frequency in doc1: {'и': 0.3333333333333333, 'ты': 0.3333333333333333, 'я': 0.3333333333333333, 'он': 0.0, 'не': 0.0, 'только': 0.0}
Term frequency in doc2: {'и': 0.3333333333333333, 'ты': 0.3333333333333333, 'я': 0.3333333333333333, 'он': 0.0, 'не': 0.0, 'только': 0.0}
Term frequency in doc3: {'и': 0.2, 'ты': 0.0, 'я': 0.6, 'он': 0.0, 'не': 0.0, 'только': 0.2}
Term frequency in doc4: {'и': 0.0, 'ты': 0.0, 'я': 0.3333333333333333, 'он': 0.0, 'не': 0.3333333333333333, 'только': 0.3333333333333333}
Term frequency in doc5: {'и': 0.0, 'ты': 0.0, 'я': 0.0, 'он': 1.0, 'не': 0.0, 'только': 0.0}


In [217]:
def computeIDF(documents):
    import math
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

idfs = computeIDF([numOfWords1, numOfWords2, numOfWords3, numOfWords4, numOfWords5])

In [218]:
def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [224]:
tfidf1 = computeTFIDF(tf1, idfs)
tfidf2 = computeTFIDF(tf2, idfs)
tfidf3 = computeTFIDF(tf3, idfs)
tfidf4 = computeTFIDF(tf4, idfs)
tfidf5 = computeTFIDF(tf5, idfs)

df = pd.DataFrame([tfidf1, tfidf2, tfidf3, tfidf4, tfidf5])
df.head()
#итоговая таблица

,и,ты,я,он,не,только
0,0.170275,0.30543,0.074381,0.000000,0.000000,0.000000
1,0.170275,0.30543,0.074381,0.000000,0.000000,0.000000
2,0.102165,0.00000,0.133886,0.000000,0.000000,0.183258
3,0.000000,0.00000,0.074381,0.000000,0.536479,0.305430
4,0.000000,0.00000,0.000000,1.609438,0.000000,0.000000


### Задание 3
Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:
а) один должен использовать CountVectorizer, другой TfidfVectorizer
б) у векторазера должны быть вручную заданы как минимум 5 параметров
в) у классификатора должно быть задано вручную как минимум 2 параметра
г) f1 мера каждого из классификаторов должна быть минимум 0.75

In [85]:
data = pd.read_csv('labeled.csv')
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [86]:
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [87]:
#логистическая регрессия с TfidfVectorizer

vectorizer = TfidfVectorizer(input='content', binary=False, ngram_range=(1,1), min_df = 5, lowercase = True)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 
y = train.toxic.values
y_test = test.toxic.values

clf = LogisticRegression(C=0.9, class_weight='balanced')
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.88      0.88      0.88       906
         1.0       0.80      0.80      0.80       536

    accuracy                           0.85      1442
   macro avg       0.84      0.84      0.84      1442
weighted avg       0.85      0.85      0.85      1442



In [95]:
# predict_proba возвращает вероятности классов
# в левой колонке вероятность 0 (нетокстичности)
# в правой - вероятность 1 (токсичности)
# логистическая регрессия
import numpy as np

probasTfIdf = clf.predict_proba(X_test)
test['probabilityTfIdf'] = [row[1] for row in probas]
test.sort_values(by='probabilityTfIdf', ascending=False)[:10]

<ipython-input-95-4359c0ba8e6d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['probabilityTfIdf'] = [row[1] for row in probas]


,index,comment,toxic,probabilityTfIdf
545,261,Ты бы и к собаке пристроился.\n,1.0,0.999130
856,6423,"Росстат сравнил уровень счастья: чувства, здор...",0.0,0.999130
13,10848,магнит ГФ покрыли краской) много денег\n,0.0,0.995522
48,4645,просто не либераст бездомный.\n,1.0,0.993006
174,9395,Я пошёл учиться на программиста в 29. Звездой ...,0.0,0.992886
1334,6738,"Это серийные устройства, используются для комм...",0.0,0.992759
651,14373,"Прощай, немытая Россия, Страна рабов, страна г...",1.0,0.992079
164,9422,Одна из причина КЗ - плохой контакт в соединен...,0.0,0.991625
1171,9517,"Обычно так и делают, просто почистили и хорош,...",0.0,0.988898
470,12931,"Вы шавки обыкновенные, которые за кость растер...",1.0,0.987354


In [96]:
# Наивный байесовский классификатор с CountVectorizer
vectorizer = CountVectorizer(input='content', binary=False, ngram_range=(1,1), min_df = 1, lowercase = True)
X1 = vectorizer.fit_transform(train.comment)
X1_test = vectorizer.transform(test.comment) 
y1 = train.toxic.values
y1_test = test.toxic.values

clf = MultinomialNB(alpha=1., fit_prior = False)
clf.fit(X1, y1)
preds = clf.predict(X1_test)

print(classification_report(y1_test, preds))

              precision    recall  f1-score   support

         0.0       0.88      0.96      0.92       981
         1.0       0.89      0.73      0.80       461

    accuracy                           0.88      1442
   macro avg       0.89      0.84      0.86      1442
weighted avg       0.89      0.88      0.88      1442



In [97]:
# predict_proba возвращает вероятности классов
# в левой колонке вероятность 0 (нетокстичности)
# в правой - вероятность 1 (токсичности)
# Наивный байесовский классификатор

probasBayes = clf.predict_proba(X1_test)
test['probabilityBayes'] = [row[1] for row in probasBayes]
test.sort_values(by='probabilityBayes', ascending=False)[:10]


<ipython-input-97-2dd3cca1630e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['probabilityBayes'] = [row[1] for row in probasBayes]


,index,comment,toxic,probabilityTfIdf,probabilityBayes
843,723,"Как известно, у Укр ины (т.е. окр ины), слепле...",1.0,0.535101,1.0
257,41,Ну давай разберём всё тобой написанное. Бляядь...,1.0,0.324793,1.0
796,14400,"Не зря, вас, хохлов, свиньями кличут. Вы и ест...",1.0,0.140872,1.0
25,14224,01:30 о встрече с Чубайсом 02:40 как Шарий защ...,1.0,0.459606,1.0
935,2781,Пошла нахуй свинья. Хохла на штык! Хороший хох...,1.0,0.263822,1.0
846,872,"Зальете шебм? Вот этот кун. Говорит, будто зая...",1.0,0.723205,1.0
870,1699,"можете назвать ХОТЬ ОДНУ блядь нацию, которая ...",1.0,0.097269,1.0
412,768,"Хоть ты и дегенерат, но ты прав. В том, что ты...",1.0,0.385785,1.0
1012,639,"Аааа, ясно как ты знаешь США, наверное по твое...",0.0,0.517819,1.0
384,1738,Гандоны - это ты и твой хозяин - пидорас-приго...,1.0,0.108654,1.0


In [ ]:
#найдите 10 самых токсичных для каждого из классификаторов. 
#Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

#Ни один текст из десяти для двух моделей не совпал. У Байеса 10 самых токсичных получились точнее (см. рисунок).  

In [106]:
Image(url="https://i.ibb.co/RyrqfYf/image.png")

In [107]:
Image(url="https://i.ibb.co/vLX0xcb/1.png")

### Задание 1 
У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя.

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [139]:
# векторайзер с razdel.tokenize

f = open('labeled.csv', 'r', encoding='UTF-8')
from razdel import tokenize

def my_tokenizer(text):
    tokens = list(tokenize(text))
    token = [t.text for t in tokens]
    return token
    
vectorizer_custom = TfidfVectorizer(tokenizer=my_tokenizer)
X_custom = vectorizer_custom.fit_transform(f)
X_custom

<27857x69451 sparse matrix of type '<class 'numpy.float64'>'
	with 435770 stored elements in Compressed Sparse Row format>

In [136]:
# дефолтный векторайзер

f = open('labeled.csv', 'r', encoding='UTF-8')

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(f)
X

<27857x68425 sparse matrix of type '<class 'numpy.float64'>'
	with 320937 stored elements in Compressed Sparse Row format>

In [137]:
data = pd.read_csv('labeled.csv')

train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [138]:
# классификатор с дефолтным векторайзером

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 
y = train.toxic.values
y_test = test.toxic.values

clf = LogisticRegression(C=0.9, class_weight='balanced')
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.89      0.89      0.89       941
         1.0       0.80      0.80      0.80       501

    accuracy                           0.86      1442
   macro avg       0.85      0.85      0.85      1442
weighted avg       0.86      0.86      0.86      1442



In [140]:
# классификатор с векторайзером с razdel.tokenize

vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 
y = train.toxic.values
y_test = test.toxic.values

clf = LogisticRegression(C=0.9, class_weight='balanced')
clf.fit(X, y)
preds = clf.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.88      0.88      0.88       941
         1.0       0.78      0.78      0.78       501

    accuracy                           0.85      1442
   macro avg       0.83      0.83      0.83      1442
weighted avg       0.85      0.85      0.85      1442



По всем метрикам дефолтный векторайзер лучше, хотя и ненамного.